In [15]:
import numpy as np
import glob
import pickle 
import csv

person_names = {0: "Andreea", 1: "Andrea", 2: "Ellis", 3: "Eli", 4: "Hong", 5: "Smitha"}
corrections_str = []

for file in glob.glob('betas/*_correction_*.p'):
    corrections_str.append(file)
    
for file_path in corrections_str:
    # Parse string to find features known and features corrected
    print "Parsing this file:", file_path
    str_list = file_path.split("_")
    method_idx = str_list.index("BETA")
    correction_idx = str_list.index("correction")
    end_idx = len(str_list)-1
    
    person_ID = person_names[int(str_list[1])]
    features_known = str_list[method_idx+1:correction_idx]
    features_corrected = str_list[correction_idx+1:-1]
    is_rational = not set(features_known).isdisjoint(features_corrected)
    
    print "Name of human:", person_ID
    print "Robot knew about these features:", features_known
    print "Human was asked to correct these features:", features_corrected  
    print "Was human rational?", is_rational
    betas = pickle.load(open(file_path, "rb"))
    print betas, "\n"


Parsing this file: betas/betas_5_B_BETA_table_coffee_correction_table_11.p
Name of human: Smitha
Robot knew about these features: ['table', 'coffee']
Human was asked to correct these features: ['table']
Was human rational? True
[[ 5.61094689  0.87431362  0.04877626]
 [ 9.802001    0.10598068  5.04760255]] 

Parsing this file: betas/betas_1_B_BETA_table_coffee_correction_accident_11.p
Name of human: Andrea
Robot knew about these features: ['table', 'coffee']
Human was asked to correct these features: ['accident']
Was human rational? False
[[  7.76657009   0.62577397   1.07699846]
 [  9.27710319   0.09490311   0.09656332]
 [ 10.68548799   0.01924351   0.02221589]] 

Parsing this file: betas/betas_4_B_BETA_table_coffee_correction_table_11.p
Name of human: Hong
Robot knew about these features: ['table', 'coffee']
Human was asked to correct these features: ['table']
Was human rational? True
[[ 7.45080185  0.18033263  0.84679334]] 

Parsing this file: betas/betas_4_B_BETA_table_coffee_correc

Was human rational? True
[[  9.26933789  11.03382755   0.03234833]] 

Parsing this file: betas/betas_0_B_BETA_table_coffee_correction_coffee_11.p
Name of human: Andreea
Robot knew about these features: ['table', 'coffee']
Human was asked to correct these features: ['coffee']
Was human rational? True
[[ 7.47304702  2.39418139  0.06016495]] 

Parsing this file: betas/betas_5_B_BETA_table_coffee_correction_human_11.p
Name of human: Smitha
Robot knew about these features: ['table', 'coffee']
Human was asked to correct these features: ['human']
Was human rational? False
[[ 5.99143696  0.00729549  0.69870252]] 

Parsing this file: betas/betas_4_B_BETA_table_coffee_correction_human_11.p
Name of human: Hong
Robot knew about these features: ['table', 'coffee']
Human was asked to correct these features: ['human']
Was human rational? False
[[  5.89776587e+00   2.79477870e-03   1.11777081e+00]] 

Parsing this file: betas/betas_5_B_BETA_table_coffee_correction_accident_11.p
Name of human: Smitha
Ro